# Nettoyer

Ce bloc-notes montre comment nettoyer toutes les ressources créées dans la série de bloc-notes précédents.

In [ ]:
import boto3
import json

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
%store -r

Veuillez noter que ces étapes dépendent du fait que chacune d'entre elles soit complètement terminée avant que l'autre ne fonctionne. Si vous recevez un message d'erreur, attendez une minute ou deux et réessayez.

## Nettoyer les campagnes

Commencez par nettoyer les campagnes. Vous devez supprimer les campagnes avant de pouvoir supprimer les versions de la solution sur lesquelles elles sont basées. 

Le code ci-dessous supprimera les campagnes que vous avez créées.

In [ ]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

## Nettoyer les traceurs d'événements

Ensuite, nettoyez tous les traceurs d'événements. Le code ci-dessous supprimera votre traceur d'événements.

In [ ]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

## Nettoyer les filtres

Nettoyez ensuite les filtres. Le code ci-dessous itère sur tous les filtres que vous avez créés et les supprime. Remarque : si vous recevez un message d'erreur concernant une campagne, ne vous inquiétez pas. Attendez 2 minutes puis réessayez la cellule ci-dessous. 

Remarque : Vous ne pourrez pas supprimer les filtres avant que la suppression de la campagne ne soit terminée, ce qui peut prendre quelques minutes

In [ ]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## Nettoyer les solutions

Nettoyez ensuite les solutions. Le code ci-dessous supprimera les solutions qui ont été créées.

In [ ]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

## Nettoyer les jeux de données

Nettoyez ensuite les jeux de données. Le code ci-dessous supprimera les jeux de données des articles et des interactions.

In [ ]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

## Nettoyer les schémas

Nettoyez ensuite les schémas. Vous créez un schéma d'interaction et un schéma d'article, puis vous les supprimez.

In [ ]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

## Nettoyer le groupe de jeux de données

Enfin, nettoyez le groupe de jeux de données :

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

## Nettoyer le compartiment S3 et le rôle IAM

Commencez par supprimer le rôle, puis videz le compartiment et supprimez-le.

In [ ]:
iam = boto3.client('iam')

Identifiez le nom du rôle que vous souhaitez supprimer.

Vous ne pouvez pas supprimer un rôle IAM auquel des politiques sont encore attachées. Donc, après avoir identifié le rôle pertinent, répertorions la liste des politiques attachées au rôle.

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

Vous devez détacher les politiques dans le résultat ci-dessus en utilisant le code ci-dessous. Répétez l'opération pour chaque politique attachée.

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

Enfin, vous devriez être en mesure de supprimer le rôle IAM.

In [ ]:
iam.delete_role(
    RoleName = role_name
)

Pour supprimer un compartiment S3, il doit d'abord être vide. La façon la plus facile de supprimer un compartiment S3 est simplement de naviguer vers S3 dans la console AWS, de supprimer les objets du compartiment, puis de supprimer le compartiment S3 lui-même.